# Metrics of matching (advanced)
Example of the more functions to plot metrics of matching, using non-standard quantities of the catalogs

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-random-data-and-add-to-catalog" data-toc-modified-id="Generate-random-data-and-add-to-catalog-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate random data and add to catalog</a></span></li><li><span><a href="#Match-catalogs" data-toc-modified-id="Match-catalogs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Match catalogs</a></span></li><li><span><a href="#Recovery-rate" data-toc-modified-id="Recovery-rate-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Recovery rate</a></span><ul class="toc-item"><li><span><a href="#Binned-plot" data-toc-modified-id="Binned-plot-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Binned plot</a></span></li><li><span><a href="#Sky-plots" data-toc-modified-id="Sky-plots-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Sky plots</a></span></li></ul></li><li><span><a href="#Distances-of-matching" data-toc-modified-id="Distances-of-matching-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Distances of matching</a></span></li><li><span><a href="#Scaling-Relations" data-toc-modified-id="Scaling-Relations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Scaling Relations</a></span></li></ul></div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pylab as plt

## Generate random data and add to catalog

In [ ]:
# For reproducibility
np.random.seed(1)

In [ ]:
from support import gen_cluster

input1, input2 = gen_cluster(ra_min=0, ra_max=30, dec_min=0, dec_max=30)

In [ ]:
from clevar import ClCatalog

tags = {
    "ra": "RA",
    "dec": "DEC",
    "z": "Z",
    "mass": "MASS",
    "mass_err": "MASS_ERR",
    "z_err": "Z_ERR",
}
c1 = ClCatalog("Cat1", data=input1, tags=tags)
c2 = ClCatalog("Cat2", data=input2, tags=tags)
# Format for nice display
for c in ("ra", "dec", "z", "z_err"):
    c1[c].info.format = ".2f"
    c2[c].info.format = ".2f"
for c in ("mass", "mass_err"):
    c1[c].info.format = ".2e"
    c2[c].info.format = ".2e"

## Match catalogs

In [ ]:
from clevar.match import ProximityMatch
from clevar.cosmology import AstroPyCosmology

match_config = {
    "type": "cross",  # options are cross, cat1, cat2
    "verbose": True,  # print matching info of all clusters to screen
    "which_radius": "max",  # Case of radius to be used, can be: cat1, cat2, min, max
    "preference": "angular_proximity",  # options are more_massive, angular_proximity or redshift_proximity
    "catalog1": {"delta_z": 0.2, "match_radius": "1 mpc"},
    "catalog2": {"delta_z": 0.2, "match_radius": "10 arcsec"},
}

cosmo = AstroPyCosmology()
mt = ProximityMatch()
mt.match_from_config(c1, c2, match_config, cosmo=cosmo)

## Recovery rate
Compute recovery rates, in the main functions they are computed in mass and redshift bins.
Here a more advanced use where different quantities can be used.
There are several ways they can be displayed:
- Single panel with multiple lines
- Multiple panels
- 2D color map

To use this, import the `ClCatalogFuncs` package from `recovery`. It contains the functions:
- `plot`
- `plot_panel`
- `plot2D`

There functions have the names of the columns as arguments, so you can use different columns available in the catalogs.

In [ ]:
from clevar.match_metrics.recovery import ClCatalogFuncs as r_cf

In [ ]:
zbins = np.linspace(0, 2, 11)
mbins = np.logspace(13, 14, 5)

### Binned plot
The recovery rates are shown as a function of `col1` in `col2` bins.
They can be displayed as a continuous line or with steps:

In [ ]:
info = r_cf.plot(
    c1,
    col1="z",
    col2="mass",
    bins1=zbins,
    bins2=mbins,
    matching_type="cross",
    legend_format=lambda x: f"10^{{{np.log10(x)}}}",
)

They can also be transposed to be shown as a function of mass in redshift bins.

In [ ]:
info = r_cf.plot_panel(
    c1,
    col1="z",
    col2="mass",
    bins1=zbins,
    bins2=mbins,
    matching_type="cross",
    label_format=lambda x: f"10^{{{np.log10(x)}}}",
)

In [ ]:
info = r_cf.plot2D(
    c1, col1="z", col2="mass", bins1=zbins, bins2=mbins, matching_type="cross", scale2="log"
)

### Sky plots
It is possible to plot the recovery rate by positions in the sky. It is done based on healpix pixelizations:

In [ ]:
info = r_cf.skyplot(c1, matching_type="cross", nside=16, ra_lim=[-10, 50], dec_lim=[-5, 35])

## Distances of matching
The main functions in `distances` can already be binned along other quantities of the catalog and do not require a more advanced use.

Nonetheless it also has a `ClCatalogFuncs` package and can be used with the same formalism:

In [ ]:
from clevar.match_metrics.distances import ClCatalogFuncs as d_cf

In [ ]:
info = d_cf.central_position(
    c1, c2, "cross", radial_bins=20, radial_bin_units="degrees", col2="z", bins2=zbins[::4]
)

In [ ]:
info = d_cf.redshift(
    c1, c2, "cross", redshift_bins=20, col2="z", bins2=zbins[::4], normalize="cat1"
)

## Scaling Relations
Here you will be able to evaluate the scaling relations of any two quantities of the matched catalogs.
Import the `ClCatalogFuncs` package from `scaling`, the functions of this package are:
- `plot`: Scaling relation of a quantity
- `plot_color`: Scaling relation of a quantity with the colors based on a 2nd quantity
- `plot_density`: Scaling relation of a quantity with the colors based on density of points
- `plot_panel`: Scaling relation of a quantity divided in panels based on a 2nd quantity
- `plot_color_panel`: Scaling relation of a quantity with the colors based on a 2nd quantity in panels based on a 3rd quantity 
- `plot_density_panel`: Scaling relation of a quantity with the colors based on density of points in panels based on a 2rd quantity
- `plot_metrics`: Metrics of quantity scaling relation.
- `plot_density_metrics`: Scaling relation of a quantity with the colors based on density of points with scatter and bias panels
- `plot_dist`: Distribution of a quantity, binned by other component in panels, and an optional secondary component in lines.
- `plot_dist_self`: Distribution of a quantity, binned by the same quantity in panels, with option for a second quantity in lines. Is is useful to compare with plot_dist results.

take the name of the quantity to be binned:

In [ ]:
from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf

In [ ]:
info = s_cf.plot(c1, c2, "cross", col="mass", xscale="log", yscale="log")

In [ ]:
info = s_cf.plot(
    c1, c2, "cross", col="mass", xscale="log", yscale="log", col_color="z", add_err=False
)

In [ ]:
info = s_cf.plot_density(
    c1,
    c2,
    "cross",
    col="mass",
    xscale="log",
    yscale="log",
    add_err=False,
    add_fit=True,
    fit_bins1=5,
    fit_log=True,
)

In [ ]:
info = s_cf.plot_panel(
    c1, c2, "cross", col="mass", xscale="log", yscale="log", col_panel="z", bins_panel=zbins[::2]
)

In [ ]:
info = s_cf.plot_panel(
    c1,
    c2,
    "cross",
    col="mass",
    xscale="log",
    yscale="log",
    col_panel="z",
    bins_panel=zbins[::2],
    col_color="z",
)

In [ ]:
info = s_cf.plot_density_panel(
    c1, c2, "cross", col="mass", xscale="log", yscale="log", col_panel="z", bins_panel=zbins[::2]
)

In [ ]:
info = s_cf.plot_metrics(c1, c2, "cross", col="z", mode="diff_z", label1="z1")

In [ ]:
info = s_cf.plot_density_metrics(c1, c2, "cross", col="z", metrics_mode="diff_z")

In [ ]:
info = s_cf.plot_density_metrics(
    c1, c2, "cross", col="mass", metrics_mode="log", scale1="log", scale2="log"
)

In [ ]:
info = s_cf.plot_density_dist(
    c1, c2, "cross", col="mass", metrics_mode="log", scale1="log", scale2="log", add_err=False
)

In [ ]:
zbins2 = np.linspace(0, 1.2, 7)[1:]
rbins2 = [1, 5.5, 25, 50, c2["mass"].max()]
bins1 = 10 ** np.histogram(np.log10(c1["mass"]), 20)[1]
info = s_cf.plot_dist(
    c1,
    c2,
    "cross",
    "mass",
    bins1=21,
    bins2=[10**13.0, 10**13.2, 10**13.5, 1e14, 1e15],
    col_aux="z",
    bins_aux=4,
    shape="line",
    log_vals=True,
    fig_kwargs={"figsize": (15, 6)},
)